In [ ]:
from utils import *

from scipy.spatial import Voronoi
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
from shapely import Polygon, box, Point

In [ ]:
bbox = box(-180, -60, 180, 85)
km = 1 / 120

In [ ]:
def get_gdf_conflict_1year(year, exclude=[]):
    shp_file = path_data / f"PSM/sample_point/conflict_{year}.shp"
    gdf_conflict_1year = gpd.read_file(shp_file)
    return gdf_conflict_1year

In [ ]:
def voronoi_polygons(points):
    coords = np.array(list(map(lambda x: (x.x, x.y), points)))
    vor = Voronoi(coords)
    regions = [vor.vertices[line] for line in vor.regions if -1 not in line and line != []]
    return [Polygon(region) for region in regions]

In [ ]:
ring5 = Point(0, 0).buffer(5 * km, quad_segs=64)
ring10 = Point(0, 0).buffer(10 * km, quad_segs=64).difference(Point(0, 0).buffer(5 * km, quad_segs=64))
ring15 = Point(0, 0).buffer(15 * km, quad_segs=64).difference(Point(0, 0).buffer(10 * km, quad_segs=64))
ring20 = Point(0, 0).buffer(20 * km, quad_segs=64).difference(Point(0, 0).buffer(15 * km, quad_segs=64))
ring25 = Point(0, 0).buffer(25 * km, quad_segs=64).difference(Point(0, 0).buffer(20 * km, quad_segs=64))
dic_ring_area = {5: ring5.area, 10: ring10.area, 15: ring15.area, 20: ring20.area, 25: ring25.area}

def cal_voronoi_ring_ratio(year):
    gdf_conflict_1year = get_gdf_conflict_1year(year)

    points = [*gdf_conflict_1year["geometry"].values, *[Point(*xy) for xy in list(box(-180, -60, 180, 85).exterior.coords)][:-1]]

    gdf_voronoi = gpd.GeoDataFrame({}, geometry=voronoi_polygons(points))
    gdf_voronoi = gpd.clip(gdf_voronoi, bbox)

    gdf_voronoi_ring_clip = gdf_voronoi.sjoin(gdf_conflict_1year)\
        .assign(
            geo_buffer5=lambda _df: _df[["x", "y"]].apply(lambda xy: Point(*xy).buffer(5 * km, quad_segs=64), axis=1),
            geo_buffer10=lambda _df: _df[["x", "y"]].apply(lambda xy: Point(*xy).buffer(10 * km, quad_segs=64), axis=1),
            geo_buffer15=lambda _df: _df[["x", "y"]].apply(lambda xy: Point(*xy).buffer(15 * km, quad_segs=64), axis=1),
            geo_buffer20=lambda _df: _df[["x", "y"]].apply(lambda xy: Point(*xy).buffer(20 * km, quad_segs=64), axis=1),
            geo_buffer25=lambda _df: _df[["x", "y"]].apply(lambda xy: Point(*xy).buffer(25 * km, quad_segs=64), axis=1),
        )\
        .assign(
            geo_ring5=lambda _df: _df["geo_buffer5"],
            geo_ring10=lambda _df: _df[["geo_buffer10", "geo_buffer5"]].apply(lambda g_: g_[0].difference(g_[1]), axis=1),
            geo_ring15=lambda _df: _df[["geo_buffer15", "geo_buffer10"]].apply(lambda g_: g_[0].difference(g_[1]), axis=1),
            geo_ring20=lambda _df: _df[["geo_buffer20", "geo_buffer15"]].apply(lambda g_: g_[0].difference(g_[1]), axis=1),
            geo_ring25=lambda _df: _df[["geo_buffer25", "geo_buffer20"]].apply(lambda g_: g_[0].difference(g_[1]), axis=1),
        )\
        .assign(
            voronoi_clip5=lambda _df: _df[["geometry", "geo_ring5"]].apply(lambda g_: g_[0].intersection(g_[1]), axis=1),
            voronoi_clip10=lambda _df: _df[["geometry", "geo_ring10"]].apply(lambda g_: g_[0].intersection(g_[1]), axis=1),
            voronoi_clip15=lambda _df: _df[["geometry", "geo_ring15"]].apply(lambda g_: g_[0].intersection(g_[1]), axis=1),
            voronoi_clip20=lambda _df: _df[["geometry", "geo_ring20"]].apply(lambda g_: g_[0].intersection(g_[1]), axis=1),
            voronoi_clip25=lambda _df: _df[["geometry", "geo_ring25"]].apply(lambda g_: g_[0].intersection(g_[1]), axis=1),
        )\
        .assign(
            voronoi_clip5_area=lambda _df: _df[[f"voronoi_clip5"]].apply(lambda x: x.area),
            voronoi_clip10_area=lambda _df: _df[[f"voronoi_clip10"]].apply(lambda x: x.area),
            voronoi_clip15_area=lambda _df: _df[[f"voronoi_clip15"]].apply(lambda x: x.area),
            voronoi_clip20_area=lambda _df: _df[[f"voronoi_clip20"]].apply(lambda x: x.area),
            voronoi_clip25_area=lambda _df: _df[[f"voronoi_clip25"]].apply(lambda x: x.area),
        )\
        .assign(
            ratio5=lambda _df: _df["voronoi_clip5_area"] / dic_ring_area[5],
            ratio10=lambda _df: _df["voronoi_clip10_area"] / dic_ring_area[10],
            ratio15=lambda _df: _df["voronoi_clip15_area"] / dic_ring_area[15],
            ratio20=lambda _df: _df["voronoi_clip20_area"] / dic_ring_area[20],
            ratio25=lambda _df: _df["voronoi_clip25_area"] / dic_ring_area[25],
        )
    return gdf_voronoi_ring_clip

In [ ]:
for year in range(2002, 2024):
    if not (out_file:=path_data / f"PSM/voronoi/ring_ratio_{year}.csv").exists():
        gdf_voronoi_ring_clip = cal_voronoi_ring_ratio(year)
        
        df_voronoi_ring_ratio = gdf_voronoi_ring_clip[["x", "y", "idx", "ratio5", "ratio10", "ratio15", "ratio20", "ratio25"]]\
            .assign(year=year)
        df_voronoi_ring_ratio.to_csv(out_file)